<a href="https://colab.research.google.com/github/cmeneses1/kMedoidsCalculator/blob/main/k_Medoids_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k-Medoids Calculator
-----------

In [77]:
from geopy import distance
import pandas as pd
import numpy as np
import requests
import json

Loading a test data file with three columns: `Name`, `Latitude` and `Longitude`.

In [6]:
%%file test.txt
Name	Latitude	Longitude
Is	43.28503	-6.75724
Beveraso	43.2781441	-6.7986945
San Emiliano	43.2574581	-6.8336809
Riodecoba	43.3073637	-6.7926056
Herías	43.3154383	-6.805375

Writing test.txt


In [75]:
name = 'test.txt'
data = pd.read_table(name)
data

,Name,Latitude,Longitude
0,Is,43.285030,-6.757240
1,Beveraso,43.278144,-6.798695
2,San Emiliano,43.257458,-6.833681
3,Riodecoba,43.307364,-6.792606
4,Herías,43.315438,-6.805375


In [94]:
# Longitude and Latitude vectors
Longitude = data.Longitude
Latitude = data.Latitude

# Number of locations
n = len(Longitude)

# Creating a convenient string for OSRM service.
lonLatString = ''
for i in range(0, n):
    lon = Longitude[i]
    lat = Latitude[i]
    lonLatString += str(lon) + ',' + str(lat) + ';'

lonLatString = lonLatString[0:-1]

# call the OSMR API
osmrString = "http://router.project-osrm.org/table/v1/driving/" + lonLatString 
r = requests.get(osmrString)

# Extracting driving time duration Matrix
durationsMatrix = 1/60 * np.matrix(json.loads(r.content)['durations'])

# Creating a matrix of geodesic distances
distancesMatrix = np.zeros((n,n))

# Calculating distances from geopy function `distance`
for i in range(0, n):
    t1 = (Latitude[i], Longitude[i])

    for j in range(0, n):
        t2 = (Latitude[j], Longitude[j])

        if i != j:
            distancesMatrix[i, j] = distance.distance(t1, t2).km
            distancesMatrix[j, i] = distancesMatrix[i, j]

In [99]:
print(durationsMatrix)
print(distancesMatrix)

[[  0.          33.91666667  51.67833333  69.40666667  73.73166667]
 [ 33.86166667   0.          85.10833333 102.83666667 107.16166667]
 [ 51.83833333  85.32166667   0.          23.36        22.05333333]
 [ 69.48833333 102.97166667  23.365        0.          19.81333333]
 [ 73.90666667 107.39        22.06833333  19.84         0.        ]]
[[0.         3.45063376 6.92042444 3.79376084 5.16414477]
 [3.45063376 0.         3.65369544 3.28364399 4.17864396]
 [6.92042444 3.65369544 0.         6.46960515 6.83891262]
 [3.79376084 3.28364399 6.46960515 0.         1.3703845 ]
 [5.16414477 4.17864396 6.83891262 1.3703845  0.        ]]
